<a href="https://colab.research.google.com/github/TheRufael/CS770-Assignments/blob/main/Assignment_Three_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Table of contents and setup
# Table of contents
# 1) Reproducibility and setup
# 2) Data load, preprocessing, and tf.data pipelines
# 3) Model builders and training utilities
# 4) Train four models (MLP no BN, MLP BN, CNN no BN, CNN BN)
# 5) Results table, learning curves, gaps, and confusion matrices

In [ ]:

# 1) Reproducibility and setup
SEED = 42  # Define a seed for reproducibility to ensure consistent results across runs.

import os, time, random # Import necessary standard libraries.
import numpy as np # Import numpy for numerical operations.
import tensorflow as tf # Import tensorflow for building and training neural networks.

from tensorflow import keras # Import keras from tensorflow as a high-level API.
from tensorflow.keras import layers, models, optimizers, utils # Import specific modules from keras.

import pandas as pd # Import pandas for data manipulation and analysis.
from sklearn.model_selection import train_test_split # Import train_test_split for splitting data.
from sklearn.metrics import confusion_matrix # Import confusion_matrix for evaluating classification models.

import matplotlib.pyplot as plt # Import matplotlib for plotting.
import seaborn as sns # Import seaborn for enhanced data visualization.

# Fix randomness across different libraries for reproducibility.
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
tf.random.set_seed(SEED)

# Prefer deterministic operations in TensorFlow when available to further enhance reproducibility.
try:
    tf.config.experimental.enable_op_determinism(True)
except Exception:
    pass

# Enable memory growth on GPU devices if available to prevent OOM errors.
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for g in gpus:
        try:
            tf.config.experimental.set_memory_growth(g, True)
        except Exception:
            pass

# Print versions of key libraries and detected GPU devices for environment information.
print("TensorFlow", tf.__version__)
try:
    import sklearn
    print("Scikit-learn", sklearn.__version__)
except Exception:
    pass
print("Pandas", pd.__version__)
print("GPU devices", tf.config.list_physical_devices("GPU"))

TensorFlow 2.19.0
Scikit-learn 1.6.1
Pandas 2.2.2
GPU devices [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
 #  Data load, preprocessing, and tf.data pipelines

# Load the Fashion-MNIST dataset directly from keras.datasets.
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize image data to the range [0, 1] and add a channel dimension.
x_train_full = (x_train_full.astype("float32") / 255.0)[..., None]
x_test       = (x_test.astype("float32") / 255.0)[..., None]

NUM_CLASSES = 10
# Convert integer labels to one-hot encoded vectors.
y_train_full_oh = utils.to_categorical(y_train_full, NUM_CLASSES)
y_test_oh       = utils.to_categorical(y_test, NUM_CLASSES)

# Split the full training data into training and validation sets (80/20 split) with stratification.
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full_oh, test_size=0.20, random_state=SEED, stratify=y_train_full
)

INPUT_SHAPE = (28, 28, 1)
BATCH_SIZE = 64
EPOCHS = 30

# Print the shapes of the resulting train, validation, and test sets.
print("Train", x_train.shape, "Val", x_val.shape, "Test", x_test.shape)

# Build tf.data pipelines for efficient data loading and preprocessing.
AUTOTUNE = tf.data.AUTOTUNE

train_ds = (tf.data.Dataset
            .from_tensor_slices((x_train, y_train))
            .shuffle(60000, seed=SEED, reshuffle_each_iteration=True)
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTOTUNE))

val_ds = (tf.data.Dataset
          .from_tensor_slices((x_val, y_val))
          .batch(BATCH_SIZE)
          .cache()
          .prefetch(AUTOTUNE))

test_ds = (tf.data.Dataset
           .from_tensor_slices((x_test, y_test_oh))
           .batch(BATCH_SIZE)
           .prefetch(AUTOTUNE))

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train (48000, 28, 28, 1) Val (12000, 28, 28, 1) Test (10000, 28, 28, 1)


In [ ]:
#  Model builders and training utilities

# MLP builder with BatchNorm toggle
def build_mlp(use_bn: bool, input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES):
    """
    Builds a Multi-Layer Perceptron (MLP) model with an option to include Batch Normalization.
    """
    x_in = layers.Input(shape=input_shape)
    x = layers.Flatten(name="flat")(x_in)

    x = layers.Dense(256, use_bias=not use_bn, name="dense1")(x)
    if use_bn:
        x = layers.BatchNormalization(name="bn1")(x) # Batch Normalization layer
    x = layers.Activation("relu", name="relu1")(x)

    x = layers.Dense(128, use_bias=not use_bn, name="dense2")(x)
    if use_bn:
        x = layers.BatchNormalization(name="bn2")(x) # Batch Normalization layer
    x = layers.Activation("relu", name="relu2")(x)

    out = layers.Dense(num_classes, activation="softmax", dtype="float32", name="head")(x)
    return models.Model(x_in, out, name=("mlp_with_bn" if use_bn else "mlp_no_bn"))

# CNN builder with BatchNorm toggle
def build_cnn(use_bn: bool, input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES):
    """
    Builds a Convolutional Neural Network (CNN) model with an option to include Batch Normalization.
    """
    x_in = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding="same", use_bias=not use_bn, name="conv1")(x_in)
    if use_bn:
        x = layers.BatchNormalization(name="bn1")(x) # Batch Normalization layer
    x = layers.Activation("relu", name="relu1")(x)
    x = layers.MaxPooling2D(2, name="pool1")(x)

    x = layers.Conv2D(64, 3, padding="same", use_bias=not use_bn, name="conv2")(x)
    if use_bn:
        x = layers.BatchNormalization(name="bn2")(x) # Batch Normalization layer
    x = layers.Activation("relu", name="relu2")(x)
    x = layers.MaxPooling2D(2, name="pool2")(x)

    x = layers.Flatten(name="flat")(x)
    x = layers.Dense(128, use_bias=not use_bn, name="dense")(x)
    if use_bn:
        x = layers.BatchNormalization(name="bn3")(x) # Batch Normalization layer
    x = layers.Activation("relu", name="relu3")(x)

    out = layers.Dense(num_classes, activation="softmax", dtype="float32", name="head")(x)
    return models.Model(x_in, out, name=("cnn_with_bn" if use_bn else "cnn_no_bn"))

# Optimizer and compile
def compile_model(model, lr=1e-3):
    """
    Compiles a Keras model with the Adam optimizer and categorical crossentropy loss.
    """
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), # Adam optimizer
                  loss="categorical_crossentropy", # Categorical crossentropy loss
                  metrics=["accuracy"])
    return model

# Train one model and time it
def train_one(model, train_ds, val_ds, epochs=EPOCHS, verbose=2):
    """
    Trains a Keras model and measures the training time.
    """
    start = time.time()
    hist = model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=verbose)
    secs = int(time.time() - start)
    return hist, secs

# Evaluate accuracy on a dataset
def evaluate_acc(model, ds):
    """
    Evaluates the accuracy of a model on a given dataset.
    """
    _, acc = model.evaluate(ds, verbose=0)
    return float(acc)

# Mean train minus val accuracy over last five epochs
def last5_gap(hist):
    """
    Calculates the mean difference between training and validation accuracy over the last 5 epochs.
    """
    a = np.array(hist.history["accuracy"][-5:])
    v = np.array(hist.history["val_accuracy"][-5:])
    return float(np.mean(a - v))

In [ ]:
# Train four models

results = [] # Initialize an empty list to store the training results.
artifacts = {} # Initialize a dictionary to store model artifacts.

# MLP without BN
tf.keras.backend.clear_session() # Clear the Keras backend session.
mlp_no_bn = build_mlp(use_bn=False) # Build MLP without BN.
compile_model(mlp_no_bn) # Compile the model.
print(mlp_no_bn.summary()) # Print summary.
hist_mlp_no_bn, t_mlp_no_bn = train_one(mlp_no_bn, train_ds, val_ds) # Train the model.
acc_test_mlp_no_bn = evaluate_acc(mlp_no_bn, test_ds) # Evaluate on test set.
results.append({
    "Model": "MLP", "BatchNorm": "No",
    "Train Acc": float(hist_mlp_no_bn.history["accuracy"][-1]),
    "Val Acc": float(hist_mlp_no_bn.history["val_accuracy"][-1]),
    "Test Acc": acc_test_mlp_no_bn,
    "Train Time (sec)": t_mlp_no_bn,
    "Params": mlp_no_bn.count_params(),
})
artifacts["mlp_no_bn"] = {"model": mlp_no_bn, "hist": hist_mlp_no_bn}

# MLP with BN
tf.keras.backend.clear_session() # Clear the Keras backend session.
mlp_bn = build_mlp(use_bn=True) # Build MLP with BN.
compile_model(mlp_bn) # Compile the model.
print(mlp_bn.summary()) # Print summary.
hist_mlp_bn, t_mlp_bn = train_one(mlp_bn, train_ds, val_ds) # Train the model.
acc_test_mlp_bn = evaluate_acc(mlp_bn, test_ds) # Evaluate on test set.
results.append({
    "Model": "MLP", "BatchNorm": "Yes",
    "Train Acc": float(hist_mlp_bn.history["accuracy"][-1]),
    "Val Acc": float(hist_mlp_bn.history["val_accuracy"][-1]),
    "Test Acc": acc_test_mlp_bn,
    "Train Time (sec)": t_mlp_bn,
    "Params": mlp_bn.count_params(),
})
artifacts["mlp_bn"] = {"model": mlp_bn, "hist": hist_mlp_bn}

# CNN without BN
tf.keras.backend.clear_session() # Clear the Keras backend session.
cnn_no_bn = build_cnn(use_bn=False) # Build CNN without BN.
compile_model(cnn_no_bn) # Compile the model.
print(cnn_no_bn.summary()) # Print summary.
hist_cnn_no_bn, t_cnn_no_bn = train_one(cnn_no_bn, train_ds, val_ds) # Train the model.
acc_test_cnn_no_bn = evaluate_acc(cnn_no_bn, test_ds) # Evaluate on test set.
results.append({
    "Model": "CNN", "BatchNorm": "No",
    "Train Acc": float(hist_cnn_no_bn.history["accuracy"][-1]),
    "Val Acc": float(hist_cnn_no_bn.history["val_accuracy"][-1]),
    "Test Acc": acc_test_cnn_no_bn,
    "Train Time (sec)": t_cnn_no_bn,
    "Params": cnn_no_bn.count_params(),
})
artifacts["cnn_no_bn"] = {"model": cnn_no_bn, "hist": hist_cnn_no_bn}

# CNN with BN
tf.keras.backend.clear_session() # Clear the Keras backend session.
cnn_bn = build_cnn(use_bn=True) # Build CNN with BN.
compile_model(cnn_bn) # Compile the model.
print(cnn_bn.summary()) # Print summary.
hist_cnn_bn, t_cnn_bn = train_one(cnn_bn, train_ds, val_ds) # Train the model.
acc_test_cnn_bn = evaluate_acc(cnn_bn, test_ds) # Evaluate on test set.
results.append({
    "Model": "CNN", "BatchNorm": "Yes",
    "Train Acc": float(hist_cnn_bn.history["accuracy"][-1]),
    "Val Acc": float(hist_cnn_bn.history["val_accuracy"][-1]),
    "Test Acc": acc_test_cnn_bn,
    "Train Time (sec)": t_cnn_bn,
    "Params": cnn_bn.count_params(),
})
artifacts["cnn_bn"] = {"model": cnn_bn, "hist": hist_cnn_bn}

Model: "mlp_no_bn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flat (Flatten)                  │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu1 (Activation)              │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu2 (Activation)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ head (Dense)                    │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 28, 28, 32)     │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 28, 28, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu1 (Activation)              │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 14, 14, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 14, 14, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu2 (Activation)              │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flat (Flatten)                  │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       401,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu3 (Activation)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ head (Dense)                    │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 422,314 (1.61 MB)

 Trainable params: 421,866 (1.61 MB)

 Non-trainable params: 448 (1.75 KB)

None
Epoch 1/30
750/750 - 8s - 11ms/step - accuracy: 0.8735 - loss: 0.3570 - val_accuracy: 0.9013 - val_loss: 0.2816
Epoch 2/30
750/750 - 3s - 4ms/step - accuracy: 0.9182 - loss: 0.2253 - val_accuracy: 0.8986 - val_loss: 0.2862
Epoch 3/30
750/750 - 3s - 4ms/step - accuracy: 0.9385 - loss: 0.1703 - val_accuracy: 0.9028 - val_loss: 0.2768
Epoch 4/30
750/750 - 3s - 4ms/step - accuracy: 0.9564 - loss: 0.1283 - val_accuracy: 0.8935 - val_loss: 0.3283
Epoch 5/30
750/750 - 3s - 4ms/step - accuracy: 0.9682 - loss: 0.0954 - val_accuracy: 0.8913 - val_loss: 0.3470
Epoch 6/30
750/750 - 3s - 4ms/step - accuracy: 0.9776 - loss: 0.0699 - val_accuracy: 0.9055 - val_loss: 0.3199
Epoch 7/30
750/750 - 3s - 4ms/step - accuracy: 0.9827 - loss: 0.0550 - val_accuracy: 0.8977 - val_loss: 0.3756
Epoch 8/30
750/750 - 3s - 4ms/step - accuracy: 0.9826 - loss: 0.0496 - val_accuracy: 0.9091 - val_loss: 0.3534
Epoch 9/30
750/750 - 3s - 4ms/step - accuracy: 0.9876 - loss: 0.0377 - val_accuracy: 0.8975 - val_loss: 0.